In [ ]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import preprocessor
import display
import classifier
import csv_file_io
import datetime


PROJECT_HOME = './'
# DRONE = 'Inspire2' # 'Bebop2', 'Spark', 'MavicPro', 'Phantom4', 'Matrice100', 'MavicAir', 'AutelEVO', 'Inspire2', 'JME', 'ParrotAnafi']
# DATASET_DIR = PROJECT_HOME + 'dataset/validation/Inspire2/Inspire2_validation_File3.wav'
RESULT_FILE_DIR = PROJECT_HOME + 'results/'
DATASET_FILE_DIR = PROJECT_HOME + 'results/dataset/'
TRAINING_DATASET_DIR = PROJECT_HOME + 'dataset/1. training/'
VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/2. validation/'
TEST_DATASET_DIR = PROJECT_HOME + 'dataset/3. test/'
TRAINING_VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/4. training_validation/'
TRAINING_VALIDATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/5. training_validation_test/'
SEGMENTATION_DATASET_DIR = PROJECT_HOME + 'dataset/6. segmentation/'
SEGMENTATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/7. segmentation(test)/'

# now = datetime.datetime.now()
# now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

N_MFCC = 80

RESULT_FILE_NAME = "[Cross-Validation-" + str(N_MFCC) + "] result_"

SAMPLE_RATE = 44100
FIG_SIZE = (15,5)

FRAME_LENGTH = 0.250 # 250 ms
FRAME_STRIDE = 0.250 # same with frame_length for 0% overlap

DIGITS = 3

b_visualizing = False

def save_cv_classification_performance(kernel: str, report_dict: dict, acc: float, date_time: datetime, standardscaler: bool):
    if standardscaler == True:
        name = '_scale(O)_'
    else:
        name = '_scale(X)_'

    file_name = RESULT_FILE_NAME.split('_')[0] + name
    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + date_time + '.csv', 'a')
    csv_file_io.write_csv_data(f, [kernel + "'s Cross-validation report"])
    csv_file_io.write_csv_data(f, ['', 'fit_time', 'score_time', 'test_score'])
    
    for i in range(0, len(report_dict['test_score'])):
        csv_file_io.write_csv_data(f, [i, report_dict['fit_time'][i], report_dict['score_time'][i],  report_dict['test_score'][i]])    
        
    csv_file_io.write_csv_data(f, ['', '', 'accuracy', acc])    
    csv_file_io.write_csv_data(f, [])

    f.close()

In [ ]:
dict_integrated_train_validation_test_signal = preprocessor.get_integrated_signal(SEGMENTATION_DATASET_DIR, type='SEGMENTATION')

# Visualizing Audio (waveform and power spectrum)
if b_visualizing == True:
    for key, val in dict_integrated_train_validation_test_signal.items():
        display.display_waveshow(val, key + "'s Training+Validation+Test Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Training+Validation+Test Signal's Power Spectrum")

In [ ]:
# Feature Extraction (MFCC)
n_fft = int(SAMPLE_RATE * FRAME_LENGTH) # length of the FFT window, frame 하나당 sample의 수 / n_fft=2048, n_fft = int(SAMPLE_RATE * FRAME_LENGTH)
hop_length = int(SAMPLE_RATE * FRAME_STRIDE) # number of samples between successive frames / hop_length=512, hop_length = int(SAMPLE_RATE * FRAME_STRIDE)
n_mels = 128
f_max = 8000

dict_melspectrogram_train_validation_test_signal = {}
for key, val in dict_integrated_train_validation_test_signal.items():    
    dict_melspectrogram_train_validation_test_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)    

dict_mfccs_train_validation_test_signal = {}
for key, val in dict_melspectrogram_train_validation_test_signal.items():
    dict_mfccs_train_validation_test_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

# print(dict_melspectrogram_train_validation_test_signal['AutelEVO'].shape)
# print(dict_mfccs_train_validation_test_signal['AutelEVO'].shape)

if b_visualizing == True:
    for key, val in dict_mfccs_train_validation_test_signal.items():
        display.display_mfccs(val,  "'s Training+Validation+Test Signal's MFCCs")  

In [ ]:
arr_train_validation_test_data = None
arr_train_validation_test_target = None

for key, val in dict_mfccs_train_validation_test_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_train_validation_test_data is None:
        arr_train_validation_test_data = arr_mfccs_data
        arr_train_validation_test_target = arr_target        
    else:       
        arr_train_validation_test_data = np.vstack((arr_train_validation_test_data, arr_mfccs_data))
        arr_train_validation_test_target = np.vstack((arr_train_validation_test_target, arr_target))

print("Training+Validation+Test data / target : ", arr_train_validation_test_data.shape, arr_train_validation_test_target.shape)
# ararr_train_validation_test_data_target = np.hstack((arr_train_validation_test_data, arr_train_validation_test_target))
# np.savetxt(DATASET_FILE_DIR + DATASET_FILE_NAME + now_date_time + '.csv', ararr_train_validation_test_data_target, delimiter=",", fmt='%s')

In [ ]:
now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

CV = 10
b_standardscaler = True

report, acc = classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("linear", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("LinearSVC", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("rbf", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("poly", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("sigmoid", report, acc, now_date_time, b_standardscaler)

In [ ]:
now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

CV = 10
b_standardscaler = False

report, acc = classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("linear", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("LinearSVC", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("rbf", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("poly", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("sigmoid", report, acc, now_date_time, b_standardscaler)